Exercise 6.1: Latent semantic analysis

In [1]:
import requests
import re
import os
import nltk
import bs4
import scipy
import numpy
import numpy.matlib
import random

import sys

# Download tagger and wordnet resources if you do not have them already
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

(a) Download the 20 Newsgroups data set from http://qwone.com/~jason/20Newsgroups/.

(b) In   this   exercise   we   consider   only   four   of   the   newsgroups:   rec.autos,   rec.motorcycles,
rec.sport.baseball,   and   rec.sport.hockey.   Process   the   documents   of   the   four   newsgroups
using the pipeline described on the lectures, including vocabulary pruning.

In [2]:
#%% Getting a list of directory contents
def gettextlist(directory_path):
    directory_textfiles=[]
    directory_nontextfiles=[]
    directory_nonfiles=[]
    # Process each item in the directory
    directory_contents=os.listdir(directory_path)
    for contentitem in directory_contents:
        temp_fullpath=os.path.join(directory_path, contentitem)
        # Non-files (e.g. subdirectories) are stored separately
        if os.path.isfile(temp_fullpath)==0:
            directory_nonfiles.append(contentitem)
        else:
            # Is this a non-text file (not ending in .txt)?
            if temp_fullpath.find('.txt')==-1:
                directory_nontextfiles.append(contentitem)
            else:
                # This is a text file
                directory_textfiles.append(contentitem)
    return(directory_textfiles,directory_nontextfiles,directory_nonfiles)

In [3]:
# %% Basic file crawler
def basicfilecrawler(directory_path):
    # Store filenames read and their text content
    num_files_read = 0
    crawled_filenames = []
    crawled_texts = []
    directory_contentlists = gettextlist(directory_path)
    # In this basic crawled we just process text files
    # and do not handle subdirectories
    # directory_textfiles=directory_contentlists[0]
    directory_textfiles = directory_contentlists[1]
    for contentitem in directory_textfiles:
        # print('Reading file:')
        # print(contentitem)
        # Open the file and read its contents
        temp_fullpath = os.path.join(directory_path, contentitem)
        temp_file = open(temp_fullpath, "r", encoding="utf-8", errors="ignore")
        temp_text = temp_file.read()
        temp_file.close()
        # Store the read filename and content
        crawled_filenames.append(contentitem)
        crawled_texts.append(temp_text)
        num_files_read = num_files_read + 1
    return (crawled_filenames, crawled_texts)

In [4]:
def read_folder(folder):
    # Read the folder contents and return the list of folder names (not including the subfolders)
    # and the list of file names
    folder_names=[]
    for root, dirs, files in os.walk(folder):
        for name in dirs:
            folder_names.append(os.path.join(root, name))
    return folder_names

In [5]:
# Read the folder contents
file_list = read_folder(
    r"C:\Users\nguye\OneDrive - TUNI.fi\Long\University\Second year\DATA.STAT.840 Statistical Methods for Text Data Analysis\data.stat.840.xblong\ex6\20_newsgroups"
)
print(file_list)

['C:\\Users\\nguye\\OneDrive - TUNI.fi\\Long\\University\\Second year\\DATA.STAT.840 Statistical Methods for Text Data Analysis\\data.stat.840.xblong\\ex6\\20_newsgroups\\alt.atheism', 'C:\\Users\\nguye\\OneDrive - TUNI.fi\\Long\\University\\Second year\\DATA.STAT.840 Statistical Methods for Text Data Analysis\\data.stat.840.xblong\\ex6\\20_newsgroups\\comp.graphics', 'C:\\Users\\nguye\\OneDrive - TUNI.fi\\Long\\University\\Second year\\DATA.STAT.840 Statistical Methods for Text Data Analysis\\data.stat.840.xblong\\ex6\\20_newsgroups\\comp.os.ms-windows.misc', 'C:\\Users\\nguye\\OneDrive - TUNI.fi\\Long\\University\\Second year\\DATA.STAT.840 Statistical Methods for Text Data Analysis\\data.stat.840.xblong\\ex6\\20_newsgroups\\comp.sys.ibm.pc.hardware', 'C:\\Users\\nguye\\OneDrive - TUNI.fi\\Long\\University\\Second year\\DATA.STAT.840 Statistical Methods for Text Data Analysis\\data.stat.840.xblong\\ex6\\20_newsgroups\\comp.sys.mac.hardware', 'C:\\Users\\nguye\\OneDrive - TUNI.fi\\Lon

In [6]:
# Create a list of the folders to be read
folders_to_read = [
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
]

# Find the indices of the folders to be read in file_list and store them in a list
indices_to_read = []
for i in range(len(file_list)):
    # Check if the folder name is in the list of folders to be read (match the last part of the path)
    if file_list[i].split("\\")[-1] in folders_to_read:
        indices_to_read.append(i)

print(indices_to_read)

print(file_list[indices_to_read[0]])
rec_autos_filenames, rec_autos_texts = basicfilecrawler(file_list[indices_to_read[0]])
rec_motorcycles_filenames, rec_motorcycles_texts = basicfilecrawler(
    file_list[indices_to_read[1]]
)
rec_sport_baseball_filenames, rec_sport_baseball_texts = basicfilecrawler(
    file_list[indices_to_read[2]]
)
rec_sport_hockey_filenames, rec_sport_hockey_texts = basicfilecrawler(
    file_list[indices_to_read[3]]
)

[7, 8, 9, 10]
C:\Users\nguye\OneDrive - TUNI.fi\Long\University\Second year\DATA.STAT.840 Statistical Methods for Text Data Analysis\data.stat.840.xblong\ex6\20_newsgroups\rec.autos


In [7]:
print(rec_autos_filenames[0:5])
print(rec_autos_texts[0:5])
print(rec_motorcycles_filenames[0:5])
print(rec_motorcycles_texts[0:5])
print(rec_sport_baseball_filenames[0:5])
print(rec_sport_baseball_texts[0:5])
print(rec_sport_hockey_filenames[0:5])
print(rec_sport_hockey_texts[0:5])

['101551', '101552', '101553', '101554', '101555']
["Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!wupost!uunet!brunix!cs.brown.edu!cs012055\nFrom: cs012055@cs.brown.edu (Hok-Chung Tsang)\nNewsgroups: rec.autos\nSubject: Re: Saturn's Pricing Policy\nMessage-ID: <1993Apr5.230808.581@cs.brown.edu>\nDate: 5 Apr 93 23:08:08 GMT\nArticle-I.D.: cs.1993Apr5.230808.581\nReferences: <C4oxwp.KKM@news.cso.uiuc.edu> <C4vIr5.L3r@shuksan.ds.boeing.com>\nSender: news@cs.brown.edu\nOrganization: Brown Computer Science Dept.\nLines: 51\n\nIn article <C4vIr5.L3r@shuksan.ds.boeing.com>, fredd@shuksan (Fred Dickey) writes:\n|> CarolinaFan@uiuc (cka52397@uxa.cso.uiuc.edu) wrote:\n|> : \tI have been active in defending Saturn lately on the net and would\n|> : like to state my full opinion on the subject, rather than just reply to others'\n|> : points.\n|> : \t\n|> : \tThe biggest problem some people seem to be having is that Saturn\n|> : Dealers make ~$2K on a car.  I think most will a

In [8]:
mycrawled_texts = (
    rec_autos_texts
    + rec_motorcycles_texts
    + rec_sport_baseball_texts
    + rec_sport_hockey_texts
)
mycrawled_filenames = (
    rec_autos_filenames
    + rec_motorcycles_filenames
    + rec_sport_baseball_filenames
    + rec_sport_hockey_filenames
)

In [9]:
mycrawled_texts[0:5]

["Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!wupost!uunet!brunix!cs.brown.edu!cs012055\nFrom: cs012055@cs.brown.edu (Hok-Chung Tsang)\nNewsgroups: rec.autos\nSubject: Re: Saturn's Pricing Policy\nMessage-ID: <1993Apr5.230808.581@cs.brown.edu>\nDate: 5 Apr 93 23:08:08 GMT\nArticle-I.D.: cs.1993Apr5.230808.581\nReferences: <C4oxwp.KKM@news.cso.uiuc.edu> <C4vIr5.L3r@shuksan.ds.boeing.com>\nSender: news@cs.brown.edu\nOrganization: Brown Computer Science Dept.\nLines: 51\n\nIn article <C4vIr5.L3r@shuksan.ds.boeing.com>, fredd@shuksan (Fred Dickey) writes:\n|> CarolinaFan@uiuc (cka52397@uxa.cso.uiuc.edu) wrote:\n|> : \tI have been active in defending Saturn lately on the net and would\n|> : like to state my full opinion on the subject, rather than just reply to others'\n|> : points.\n|> : \t\n|> : \tThe biggest problem some people seem to be having is that Saturn\n|> : Dealers make ~$2K on a car.  I think most will agree with me that the car is\n|> : comparably price

In [10]:
# Exclude header lines from each message
excludedlinemarkers = [
    "Xref:",
    "Path:",
    "From:",
    "Newsgroups:",
    "Subject:",
    "Summary:",
    "Keywords:",
    "Message-ID:",
    "Date:",
    "Expires:",
    "Followup-To:",
    "Distribution:",
    "Organization:",
    "Approved:",
    "Supersedes:",
    "Lines:",
    "NNTP-Posting-Host:",
    "References:",
    "Sender:",
    "In-Reply-To:",
    "Article-I.D.:",
    "Reply-To:",
    "Nntp-Posting-Host:",
]
for k in range(len(mycrawled_texts)):
    templines = mycrawled_texts[k].splitlines()
    remaininglines = []
    for l in range(len(templines)):
        line_should_be_excluded = 0
        for m in range(len(excludedlinemarkers)):
            if len(templines[l]) >= len(excludedlinemarkers[m]):
                if (
                    excludedlinemarkers[m]
                    == templines[l][0 : len(excludedlinemarkers[m])]
                ):
                    line_should_be_excluded = 1
                    break
        if line_should_be_excluded == 0:
            remaininglines.append(templines[l])
    mycrawled_texts[k] = "\n".join(remaininglines)

In [11]:
print(mycrawled_texts[0:5])

["\nIn article <C4vIr5.L3r@shuksan.ds.boeing.com>, fredd@shuksan (Fred Dickey) writes:\n|> CarolinaFan@uiuc (cka52397@uxa.cso.uiuc.edu) wrote:\n|> : \tI have been active in defending Saturn lately on the net and would\n|> : like to state my full opinion on the subject, rather than just reply to others'\n|> : points.\n|> : \t\n|> : \tThe biggest problem some people seem to be having is that Saturn\n|> : Dealers make ~$2K on a car.  I think most will agree with me that the car is\n|> : comparably priced with its competitors, that is, they aren't overpriced \n|> : compared to most cars in their class.  I don't understand the point of \n|> : arguing over whether the dealer makes the $2K or not?  \n|> \n|> I have never understood what the big deal over dealer profits is either.\n|> The only thing that I can figure out is that people believe that if\n|> they minimize the dealer profit they will minimize their total out-of-pocket\n|> expenses for the car. While this may be true in some cases,

In [12]:
# %% Tokenize loaded texts and change them to NLTK format
mycrawled_nltktexts = []
for k in range(len(mycrawled_texts)):
    temp_tokenizedtext = nltk.word_tokenize(mycrawled_texts[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mycrawled_nltktexts.append(temp_nltktext)

In [13]:
mycrawled_nltktexts

[<Text: In article < C4vIr5.L3r @ shuksan.ds.boeing.com > ,...>,
 <Text: In article < 1993Apr5.135153.11132 @ wdl.loral.com > gwm...>,
 <Text: THANKS TO ALL OF YOU WHO RESPONDED TO...>,
 <Text: The subject says it all . My 1984...>,
 <Text: wolfson @ regatta.sps.mot.com ( Stephen Wolfson ) writes...>,
 <Text: : TIN [ version 1.1 PL8.8>,
 <Text: I 'm looking for a replacement radio/tape player...>,
 <Text: I am thinking about getting an Infiniti G20...>,
 <Text: In article < 1993Apr5.175719.7892 @ telxon.mis.telxon.com > joes...>,
 <Text: In article < 1993Apr5.212645.15988 @ nntpd.lkg.dec.com > kenyon...>,
 <Text: In article < 3mwF2B1w165w @ njcc.wisdom.bubble.org > ,...>,
 <Text: Do n't have a list of what 's...>,
 <Text: In article < 1993Apr3.005245.10615 @ michael.apple.com > ems...>,
 <Text: In article < 1pq29p $ 29p @ seven-up.East.Sun.COM...>,
 <Text: News-Software : VAX/VMS VNEWS 1.41 In article <...>,
 <Text: This past winter I drove from NYC to...>,
 <Text: Sayeth `` Joseph D. 

In [14]:
# %% Make all crawled texts lowercase
mycrawled_lowercasetexts = []
for k in range(len(mycrawled_nltktexts)):
    temp_lowercasetext = []
    for l in range(len(mycrawled_nltktexts[k])):
        lowercaseword = mycrawled_nltktexts[k][l].lower()
        temp_lowercasetext.append(lowercaseword)
    temp_lowercasetest = nltk.Text(temp_lowercasetext)
    mycrawled_lowercasetexts.append(temp_lowercasetext)

In [15]:
# %% Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag = -1
    if postag[0] == "N":
        wordnettag = "n"
    elif postag[0] == "V":
        wordnettag = "v"
    elif postag[0] == "J":
        wordnettag = "a"
    elif postag[0] == "R":
        wordnettag = "r"
    return wordnettag

In [16]:
# %% POS tag and lemmatize the loaded texts

lemmatizer = nltk.stem.WordNetLemmatizer()


def lemmatizetext(nltktexttolemmatize):
    # Tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # Lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # Store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return lemmatizedtext


mycrawled_lemmatizedtexts = []
for k in range(len(mycrawled_lowercasetexts)):
    lemmatizedtext = lemmatizetext(mycrawled_lowercasetexts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mycrawled_lemmatizedtexts.append(lemmatizedtext)

In [17]:
# %% Find the vocabulary, in a distributed fashion
myvocabularies = []
myindices_in_vocabularies = []
# Find the vocabulary of each document
for k in range(len(mycrawled_lemmatizedtexts)):
    # Get unique words and where they occur
    temptext = mycrawled_lemmatizedtexts[k]
    uniqueresults = numpy.unique(temptext, return_inverse=True)
    uniquewords = uniqueresults[0]
    wordindices = uniqueresults[1]
    # Store the vocabulary and indices of document words in it
    myvocabularies.append(uniquewords)
    myindices_in_vocabularies.append(wordindices)

In [18]:
# Unify the vocabularies.
# First concatenate all vocabularies
tempvocabulary = []
for k in range(len(mycrawled_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])
# Find the unique elements among all vocabularies
uniqueresults = numpy.unique(tempvocabulary, return_inverse=True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]
# Translate previous indices to the unified vocabulary.
# Must keep track where each vocabulary started in
# the concatenated one.
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mycrawled_lemmatizedtexts)):
    # In order to shift word indices, we must temporarily
    # change their data type to a Numpy array
    tempindices = numpy.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])

In [19]:
# %% Count the numbers of occurrences of each unique word
# Let's count also various statistics over the documents
unifiedvocabulary_totaloccurrencecounts = numpy.zeros((len(unifiedvocabulary), 1))
unifiedvocabulary_documentcounts = numpy.zeros((len(unifiedvocabulary), 1))
unifiedvocabulary_meancounts = numpy.zeros((len(unifiedvocabulary), 1))
unifiedvocabulary_countvariances = numpy.zeros((len(unifiedvocabulary), 1))

In [20]:
# First pass: count occurrences
for k in range(len(mycrawled_lemmatizedtexts)):
    occurrencecounts = numpy.zeros((len(unifiedvocabulary), 1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]] = (
            occurrencecounts[myindices_in_unifiedvocabulary[k][l]] + 1
        )
    unifiedvocabulary_totaloccurrencecounts = (
        unifiedvocabulary_totaloccurrencecounts + occurrencecounts
    )
    unifiedvocabulary_documentcounts = unifiedvocabulary_documentcounts + (
        occurrencecounts > 0
    )

In [22]:
# %% Inspect frequent words
# Sort words by largest total (or mean) occurrence count
highest_totaloccurrences_indices = numpy.argsort(
    -1 * unifiedvocabulary_totaloccurrencecounts, axis=0
)
print(numpy.squeeze(unifiedvocabulary[highest_totaloccurrences_indices[0:100]]))
print(
    numpy.squeeze(
        unifiedvocabulary_totaloccurrencecounts[highest_totaloccurrences_indices[0:100]]
    )
)

['>' 'the' ',' '.' '--' 'be' '*' 'a' 'to' 'i' ')' 'in' 'and' '(' 'of' ':'
 '@' 'have' 'that' 'it' 'you' '!' 'for' 'do' '?' 'on' '|' "'s" '-' "n't"
 'this' "''" 'with' 'but' 'he' 'not' '0' 'they' '1' 'as' '<' 'at' 'if'
 'get' '``' 'write' 'or' 'my' '...' '2' 'article' 'game' 'go' 'would' '#'
 'can' 'about' 'one' 'will' 'all' 'there' 'an' 'what' 'from' 'out' 'so'
 'good' 'car' 'like' 'year' 'think' 'by' 'me' 'team' 'up' 'his' '%' 'just'
 'when' '3' 'more' 'make' 'your' 'no' 'know' 'any' 'say' 'who' 'than' '4'
 'we' 'time' 'some' 'play' 'well' 'see' 'how' 'player' 'win' 'only']
[40640. 40032. 39497. 39110. 35859. 27296. 20092. 17910. 17358. 15952.
 14883. 14110. 14036. 13692. 13180. 11444.  9901.  9477.  9199.  8716.
  7144.  7079.  7017.  6890.  6686.  6048.  6030.  5907.  4914.  4911.
  4768.  4515.  4514.  4288.  4140.  4041.  3977.  3925.  3803.  3753.
  3648.  3569.  3533.  3267.  3265.  3261.  3241.  3225.  2855.  2854.
  2745.  2728.  2696.  2671.  2518.  2416.  2392.  2376.  2314.

In [23]:
len(unifiedvocabulary)

42038

In [24]:
#%% Vocabulary pruning
nltkstopwords=nltk.corpus.stopwords.words('english')
pruningdecisions=numpy.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k]=1
    # Rule 2: if the word is in the top 1% of frequent words
    if (k in highest_totaloccurrences_indices[\
        0:int(numpy.floor(len(unifiedvocabulary)*0.01))]):
        pruningdecisions[k]=1
    # Rule 3: if the word is too short
    if len(unifiedvocabulary[k])<3:
        pruningdecisions[k]=1
    # Rule 4: if the word is too long
    if len(unifiedvocabulary[k])>20:
        pruningdecisions[k]=1
    # Rule 5: if the word has unwanted characters
    # (here for simplicity only a-z allowed)
    # if unifiedvocabulary[k].isalpha()==False:
    #     pruningdecisions[k]=1
    # Rule 7: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1

In [25]:
#%% Inspect remaining frequent words
# Sort remaining words by largest total (or mean) occurrence count
remainingindices=numpy.squeeze(numpy.where(pruningdecisions==0)[0])
remainingvocabulary=unifiedvocabulary[remainingindices]
remainingvocabulary_totaloccurrencecounts= \
    unifiedvocabulary_totaloccurrencecounts[remainingindices]
remaining_highest_totaloccurrences_indices= \
    numpy.argsort(-1*remainingvocabulary_totaloccurrencecounts,axis=0)
print(numpy.squeeze(remainingvocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(numpy.squeeze(remainingvocabulary_totaloccurrencecounts[ \
    remaining_highest_totaloccurrences_indices[0:100]]))

['wheel' 'yet' 'driver' 'live' 'model' 'able' 'bit' 'ago' 'city' 'cost'
 'total' 'mind' 'mention' 'man' 'brave' 'ford' 'penalty' 'final' 'large'
 'anyway' 'bill' 'idea' 'coach' 'saw' 'performance' 'beat' 'version'
 'montreal' 'rule' 'group' 'ice' 'seat' 'face' 'hitter' 'friend'
 'although' 'almost' 'today' 'order' 'comment' 'bos' 'minute' 'stats'
 'hold' 'det' 'follow' 'american' 'insurance' 'job' 'month' 'walk' 'local'
 'past' 'smith' 'joe' 'honda' 'easy' 'tie' 'cal' 'news' 'hell' 'ticket'
 'x-newsreader' 'wait' 'bring' 'hope' 'jet' 'helmet' 'stuff' 'morris'
 'rider' 'left' 'word' 'compare' 'van' 'note' 'experience' 'york' 'center'
 'information' 'flyer' 'add' 'rear' 'pen' 'puck' 'contact' 'set' 'defense'
 'young' 'break' 'netcom.com' 'others' 'claim' 'later' 'e-mail' 'design'
 'matter' 'late' 'whether' 'tor']
[254. 253. 251. 251. 250. 249. 248. 247. 246. 246. 245. 245. 244. 244.
 243. 241. 240. 240. 239. 237. 232. 232. 230. 229. 229. 228. 225. 224.
 223. 222. 222. 222. 221. 221. 221.

In [26]:
# %% Get indices of documents to remaining words
oldtopruned = []
tempind = -1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k] == 0:
        tempind = tempind + 1
        oldtopruned.append(tempind)
    else:
        oldtopruned.append(-1)

# %% Create pruned texts

mycrawled_prunedtexts = []
myindices_in_prunedvocabulary = []
for k in range(len(mycrawled_lemmatizedtexts)):
    temp_newindices = []
    temp_newdoc = []
    for l in range(len(mycrawled_lemmatizedtexts[k])):
        temp_oldindex = myindices_in_unifiedvocabulary[k][l]
        temp_newindex = oldtopruned[temp_oldindex]
        if temp_newindex != -1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    mycrawled_prunedtexts.append(temp_newdoc)
    myindices_in_prunedvocabulary.append(temp_newindices)

(c) Create a TF-IDF representation for the documents, using Length-normalized frequency (TF)
and Smoothed logarithmic inverse document frequency (IDF).

In [27]:
# %% Create TF-IDF vectors
n_docs = len(mycrawled_prunedtexts)
n_vocab = len(remainingvocabulary)
# Matrix of term frequencies
tfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
# Row vector of document frequencies
dfvector = scipy.sparse.lil_matrix((1, n_vocab))
# Loop over documents
for k in range(n_docs):
    # Row vector of which words occurred in this document
    temp_dfvector = scipy.sparse.lil_matrix((1, n_vocab))
    # Loop over words
    for l in range(len(mycrawled_prunedtexts[k])):
        # Add current word to term-frequency count and document-count
        currentword = myindices_in_prunedvocabulary[k][l]
        tfmatrix[k, currentword] = tfmatrix[k, currentword] + 1
        temp_dfvector[0, currentword] = 1
    # Add which words occurred in this document to overall document counts
    dfvector = dfvector + temp_dfvector

In [28]:
# length-normalized frequency for the TF part
for i in range(n_docs):
    for j in range(len(tfmatrix.data[i])):
        tfmatrix.data[i][j] = tfmatrix.data[i][j] / len(mycrawled_prunedtexts[i])

In [29]:
# Let's use raw term count, and smoothed logarithmic idf
idfvector = numpy.squeeze(numpy.array(dfvector.todense()))
idfvector=1+numpy.log(((idfvector+1)**-1)*n_docs)
# Use the count statistics to compute the tf-idf matrix
tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
for k in range(n_docs):
    # Combine the tf and idf terms
    tempindices = numpy.nonzero(tfmatrix[k, :])[1]
    tfterm = numpy.squeeze(numpy.array(tfmatrix[k, tempindices].todense()))
    tfidfmatrix[k, tempindices] = tfterm * idfvector[tempindices]
    


(d) Apply latent semantic analysis to the TF-IDF matrix, to find 10 underlying factors


In [30]:

# Compute 10 factors from LSA
n_low_dimensions = 10
Uleft, D, UrightT = scipy.sparse.linalg.svds(tfidfmatrix.todense(), k=n_low_dimensions)

In [31]:
# Examine the singular values
# print(D)
# Examine a factor (here the one with largest singular value)
# print(UrightT)

[4.94222698 4.95987316 5.07631109 5.11003131 5.22036183 5.63519183
 5.9375771  6.68093678 7.36644399 7.78621439]


In [33]:
# 10 words with largest absolute weights in each factor
for i in range(UrightT.shape[0]):
    print('10 words with largest absolute weights in factor ' + str(i+1))
    print('-------------------------------------------------------------')
    topweights_indices=numpy.argsort(-1*numpy.abs(UrightT[i,:]))
    print(remainingvocabulary[topweights_indices[0:10]])
    print('\n')

10 words with largest absolute weights in factor 1
-------------------------------------------------------------
['osf.org' 'research.nj.nec.com' 'tin' 'x-newsreader' '1.1' 'version'
 'group' 'pl8' 'appreciate' 'honda']


10 words with largest absolute weights in factor 2
-------------------------------------------------------------
['tin' 'x-newsreader' '1.1' 'version' 'pl8' 'osf.org'
 'research.nj.nec.com' 'appreciate' 'group' 'pl4']


10 words with largest absolute weights in factor 3
-------------------------------------------------------------
['altima' 'generic' 'jewish' 'kingman' 'appreciate' 'stanza' 'nissan'
 'group' 'paperwork' 'nameplate']


10 words with largest absolute weights in factor 4
-------------------------------------------------------------
['jewish' 'kingman' 'tin' 'x-newsreader' '1.1' 'vb30'
 'lafibm.lafayette.edu' 'appreciate' 'acad.drake.edu' 'version']


10 words with largest absolute weights in factor 5
------------------------------------------------------

There are some words, which is realted to the main subject of the directory.
For example:

rec.autos: honda, pl8, nissan, kingman, altima, tesla.njit.edu, license, 1993apr20.102857.1...
rec.motorcycles: nissan, honda, kingman, cub, rider.cactus.org, license...
rec.sport.baseball: captain, career, group, baseman,...
rec.sport.hockey: career, dream, group,...

(f) Do the same with 15 factors (the first 10 factors will be the same). Do the new 5 factors seem more or less meaningful?

In [34]:
# Compute 15 factors from LSA
n_low_dimensions = 15
Uleft, D, UrightT = scipy.sparse.linalg.svds(tfidfmatrix.todense(), k=n_low_dimensions)

In [35]:
# 10 words with largest absolute weights in each factor
for i in range(UrightT.shape[0]):
    print('10 words with largest absolute weights in factor ' + str(i+1))
    print('-------------------------------------------------------------')
    topweights_indices=numpy.argsort(-1*numpy.abs(UrightT[i,:]))
    print(remainingvocabulary[topweights_indices[0:10]])
    print('\n')

10 words with largest absolute weights in factor 1
-------------------------------------------------------------
['arocha' '=-=' 'eliot' 'jeffries' 'status' 'yankee' 'blah' 'stats'
 'thank' 'dolven']


10 words with largest absolute weights in factor 2
-------------------------------------------------------------
['honda' 'subscribe' 'group' 'tour' 'lake' 'forget' 'final' 'blah' 'stats'
 'advantage']


10 words with largest absolute weights in factor 3
-------------------------------------------------------------
['group' 'lake' 'final' 'maine' 'advantage' '5-4' 'lssu' 'blah'
 'appreciate' 'terry']


10 words with largest absolute weights in factor 4
-------------------------------------------------------------
['advantage' 'lake' 'final' 'group' 'maine' '5-4' 'lssu' 'honda' 'terry'
 'blah']


10 words with largest absolute weights in factor 5
-------------------------------------------------------------
['advantage' 'group' 'lake' 'final' 'honda' 'maine' 'subscribe' '5-4'
 'blah' 'lss

The new 5 factors seem less meaningful as they are overlapping with the first 10 factors and fewer words are related to the main subject of the directory.

Exercise 6.2: Probabilistic latent semantic analysis

(a) Using the same data as in Exercise 6.1 (four newsgroups), create a term frequency matrix of raw term counts for the documents.

Done above!

(b) Apply PLSA to the term frequency matrix to find 10 underlying factors.

In [49]:
import numpy, numpy.matlib, scipy, scipy.stats
def plsa(document_to_word_matrix, n_topics, n_iterations):    
    n_docs=numpy.shape(document_to_word_matrix)[0]            # Number of documents and vocabulary words
    n_vocab=numpy.shape(document_to_word_matrix)[1]       
    theta = scipy.stats.uniform.rvs(size=(n_vocab,n_topics))  # Prob of words per topic: random init
    theta = theta/numpy.matlib.repmat(numpy.sum(theta,axis=0),n_vocab,1)    
    psi = scipy.stats.uniform.rvs(size=(n_topics,n_docs))     # Probs topics per document: random init
    psi = psi/numpy.matlib.repmat(numpy.sum(psi,axis=0),n_topics,1)    
    n_words_in_docs = numpy.squeeze(numpy.array(             # Numbers of words in documents: computed once
        numpy.sum(document_to_word_matrix,axis=1)))    
    n_totalwords = numpy.sum(n_words_in_docs)                 # Total number of words: computed once    
    pi = n_words_in_docs/n_totalwords                         # Document probs: computed once        
    for myiter in range(n_iterations):                        # Perform Expectation-Maximization iterations
        # ===Perform E-step====
        doc_word_to_topics = []               # Compute theta_{v|t}psi_{t|d}/sum_t' theta_{v|t'}psi_{t'|d} 
        doc_word_to_topic_sum = numpy.zeros((n_docs,n_vocab))               
        for t in range(n_topics):
            doc_word_to_topict = \
                numpy.matlib.repmat(theta[:,t],n_docs,1) * \
                numpy.matlib.repmat(psi[t,:],n_vocab,1).T             
            myepsilon=1e-14           # Add a positive number to avoid divisions by zero
            doc_word_to_topict += myepsilon 
            doc_word_to_topics.append(doc_word_to_topict)
            doc_word_to_topic_sum += doc_word_to_topict
        # Print the progress of the algorithm every 5% of iterations
        if numpy.mod(myiter,numpy.floor(n_iterations/20))==0:
            print('Iteration ' + str(myiter) + ' of ' + str(n_iterations))
        for t in range(n_topics):
            doc_word_to_topics[t] /= doc_word_to_topic_sum 
        # =======Perform M-step=======                        
        # Add a small number to word counts to avoid divisions by zero
        for t in range(n_topics):                            # Compute document-to-topic probabilities.
            psi[t,:] = numpy.squeeze(numpy.array(numpy.sum( \
               numpy.multiply(document_to_word_matrix+myepsilon,doc_word_to_topics[t]),axis=1)))        
        psi /= numpy.matlib.repmat(numpy.sum(psi,axis=0),n_topics,1)        
        for t in range(n_topics):                            # Compute topic-to-word probabilities
            theta[:,t]= numpy.squeeze(numpy.array(numpy.sum( \
                 numpy.multiply(document_to_word_matrix,doc_word_to_topics[t]),axis=0).T))       
        theta /= numpy.matlib.repmat(numpy.sum(theta,axis=0),n_vocab,1)
    return(pi,psi,theta)

In [50]:
#%% Try PLSA on a very small data set
# Let's take the same 500 features as in LSI, 
# and the same 1000 baseball documents but use
# the term-frequency (TF) values for the features
dimensiontotals=numpy.squeeze(numpy.array(numpy.sum(tfidfmatrix,axis=0)))
highesttotals=numpy.argsort(-1*dimensiontotals)
Xsmall=tfmatrix[:,highesttotals[0:500]].todense()

# Due to the progress is really slow, we will limit Xsmall
# Run PLSA
n_topics=10
n_iterations=200
pi,psi,theta=plsa(Xsmall, n_topics, n_iterations)

Iteration 0 of 200
Iteration 10 of 200
Iteration 20 of 200
Iteration 30 of 200
Iteration 40 of 200
Iteration 50 of 200
Iteration 60 of 200
Iteration 70 of 200
Iteration 80 of 200
Iteration 90 of 200
Iteration 100 of 200
Iteration 110 of 200
Iteration 120 of 200
Iteration 130 of 200
Iteration 140 of 200
Iteration 150 of 200
Iteration 160 of 200
Iteration 170 of 200
Iteration 180 of 200
Iteration 190 of 200


In [51]:
# Examine the factor probabilities p(t) = sum_d p(t|d) p(d)
print(numpy.sum(psi*numpy.matlib.repmat(pi,n_topics,1),axis=1))

[0.10072331 0.11888919 0.1144711  0.09718951 0.09335478 0.08529077
 0.09806643 0.08731324 0.09328984 0.11141181]


In [52]:
# Examine a factor (here the one with largest probability, factor 1)
print(theta[:,1])

[8.23519449e-14 3.61921970e-14 5.51155102e-14 8.92854254e-14
 2.68913444e-14 4.68117189e-14 2.22540615e-14 6.29719086e-14
 8.21385373e-14 2.05973091e-14 6.80604126e-03 4.07289393e-14
 3.49066954e-03 5.91797288e-14 9.33565208e-14 2.91972735e-02
 8.06864014e-03 6.69046476e-14 3.93836766e-14 1.80921007e-02
 2.63238919e-02 1.87210794e-13 2.30226157e-13 5.50440314e-14
 6.31047409e-14 1.37492096e-13 5.59832019e-14 2.50968681e-03
 5.74334758e-14 4.29524364e-14 1.19586598e-02 6.38241939e-14
 3.75930694e-14 2.77888024e-14 8.87490917e-14 5.30363440e-03
 1.41782277e-13 4.90522970e-14 5.57111703e-14 1.71061811e-13
 1.44548989e-14 8.39292098e-03 7.97421553e-14 7.60799880e-13
 1.03118091e-12 1.29038344e-13 6.21256602e-14 6.58959030e-14
 5.90213397e-03 1.29328147e-13 6.60231477e-14 1.73968286e-13
 5.35671718e-14 1.16596851e-02 4.33086614e-14 1.22426050e-13
 8.14149267e-14 3.02502059e-13 1.91406945e-13 1.04255786e-13
 3.05050952e-13 1.17067091e-02 1.95935906e-02 4.36587948e-14
 7.20427193e-14 1.030569

In [53]:
# Words with largest absolute weights in the factor
topweights_indices=numpy.argsort(-1*numpy.abs(theta[:,1]))
print(remainingvocabulary[highesttotals[topweights_indices[0:20]]])

['model' 'ticket' 'bring' 'rider' 'stuff' 'love' 'possible' 'pull' 'star'
 'saw' 'delete' 'folk' 'suck' 'stand' 'school' 'penalty' 'follow'
 'information' 'exactly' 'learn']


In [54]:
# Same for the next biggest factor (here factor 4)
topweights_indices=numpy.argsort(-1*numpy.abs(theta[:,4]))
print(remainingvocabulary[highesttotals[topweights_indices[0:20]]])

['michael' 'man' 'group' 'news' 'city' 'wave' 'phillies' 'scott' 'already'
 'dead' 'york' 'internet' 'radio' 'station' 'philadelphia' 'tie' 'field'
 'charles' 'cage' 'green']


(c) Describe the resulting factors: list the 10 words with highest probability in each factor. 

In [67]:
# (c) Describe the resulting factors: list the 10 words with highest probability in each factor. 
for i in range(theta.shape[0]):
    print('10 words with highest probability in factor ' + str(i+1))
    print('-------------------------------------------------------------')
    topweights_indices=numpy.argsort(-1*theta[i,:])
    print(remainingvocabulary[highesttotals[topweights_indices[0:10]]])
    
    print('\n')

10 words with highest probability in factor 1
-------------------------------------------------------------
['tin' 'stats' 'appreciate' 'version' '1.1' 'x-newsreader' 'email' 'test'
 'honda' 'e-mail']


10 words with highest probability in factor 2
-------------------------------------------------------------
['honda' 'version' 'stats' 'email' 'appreciate' 'tin' 'x-newsreader'
 'e-mail' 'test' '1.1']


10 words with highest probability in factor 3
-------------------------------------------------------------
['stats' 'tin' 'version' 'appreciate' 'x-newsreader' 'honda' 'email'
 'e-mail' 'test' '1.1']


10 words with highest probability in factor 4
-------------------------------------------------------------
['email' 'appreciate' 'stats' 'test' 'version' 'x-newsreader' 'honda'
 '1.1' 'e-mail' 'tin']


10 words with highest probability in factor 5
-------------------------------------------------------------
['tin' 'version' 'stats' 'x-newsreader' 'honda' 'email' 'e-mail'
 'appreciate' '

The resulting factors are related to the topic of each directory, as the same as exercise 6.1

(d) Find, for each factor, the document (message) with highest probability of that factor, and print its 100 first words.,

In [69]:
# 10 words with largest absolute weights in each factor
for i in range(psi.shape[0]):
    topweights_indices = numpy.argsort(-1 * numpy.abs(psi[i, :]))
    print('Document index with highest factor probability: ' + str(topweights_indices[0] + 1))
    print('First 100 words of factor ' + str(i+1) + ' of the document with highest probability')
    print(remainingvocabulary[topweights_indices[0:100]])
    print('\n')

Document index with highest factor probability: 2937
First 100 words of factor 1 of the document with highest probability
['buchberger' 'bunt' 'cuyler' 'amazing' 'colonial' 'checker' 'concession'
 'bum' 'challenging' 'colored' 'closed-minded' 'camping'
 'cms.cc.wayne.edu' "'57" 'compulsive' 'bythe' 'compromise' 'beast'
 'competitive' 'chess.ncsu.edu' 'anonymous' '16,005' 'c4zcii.ftn'
 'cluster' 'breakaway' 'chassis' 'anthony' 'cannucks' 'concern' 'barberie'
 'antenna' 'crusade' '185900001' 'basement' 'att.com' 'contractual'
 'canadiens' 'ching' 'cam' 'chi-5' '.533' '05-04' '1110' 'competitor'
 'ala' 'claude' 'boever' 'collegiate' 'creighton' '12:40' 'alternator'
 'conceivable' 'boyle' 'clockwise' 'cmk' 'bogey' '1988' 'csc.albany.edu'
 'dagibbs' 'cmuvm.csv.cmich.edu' 'd6s' '15th' 'd.sweeney' 'cgkarras'
 'burke' 'business' '19:59' '04-03' 'cream' 'copy' '7:30' 'con' '1138'
 'bald' '===' '094' 'appoint' '987' '0.571' 'casey' 'charm' '1.8' 'aug'
 'cyt' 'complaint' 'closeup' 'cpsc.ucalgary.

(e) Do the factors seem related to individual newsgroups? Does their content seem meaningful?

No, the factors seem not related to individual newsgroups. The content seems not meaningful.

(f) Optional: let's see if an AI chatbot can analyze these factors. See exercise 2.4 regarding possible chatbots; report what chatbot you used. 

Take one of the factors, e.g., the highest probability one, and give its list of top words to an AI chatbot; you can tell it that the factor has been learned from the four newsgroups. Ask it to interpret the factor based on the word list. Ask it also to generate a text that could be drawn from the factor. Does its interpretation seem meaningful and likely correct?

Question: 

Document index with highest factor probability: 2937
First 100 words of factor 1 of the document with highest probability
['buchberger' 'bunt' 'cuyler' 'amazing' 'colonial' 'checker' 'concession'
 'bum' 'challenging' 'colored' 'closed-minded' 'camping'
 'cms.cc.wayne.edu' "'57" 'compulsive' 'bythe' 'compromise' 'beast'
 'competitive' 'chess.ncsu.edu' 'anonymous' '16,005' 'c4zcii.ftn'
 'cluster' 'breakaway' 'chassis' 'anthony' 'cannucks' 'concern' 'barberie'
 'antenna' 'crusade' '185900001' 'basement' 'att.com' 'contractual'
 'canadiens' 'ching' 'cam' 'chi-5' '.533' '05-04' '1110' 'competitor'
 'ala' 'claude' 'boever' 'collegiate' 'creighton' '12:40' 'alternator'
 'conceivable' 'boyle' 'clockwise' 'cmk' 'bogey' '1988' 'csc.albany.edu'
 'dagibbs' 'cmuvm.csv.cmich.edu' 'd6s' '15th' 'd.sweeney' 'cgkarras'
 'burke' 'business' '19:59' '04-03' 'cream' 'copy' '7:30' 'con' '1138'
 'bald' '===' '094' 'appoint' '987' '0.571' 'casey' 'charm' '1.8' 'aug'
 'cyt' 'complaint' 'closeup' 'cpsc.ucalgary.ca' 'commute' 'am2x+'
 'collie/golden' 'ame' 'alright' 'cvax.psi.ch' 'center' 'comfortable'
 'athena.mit.edu' 'adjustable' '.653' '0.421' 'card']

Can you tell the factor has  been learned from ec.autos, rec.motorcycles,
rec.sport.baseball, and rec.sport.hockey? 

interpret the factor based on the word list

generate a text that could be
drawn from the factor

CHAT GPT RESPONSE:

Based on the provided information, it seems like the document index with the highest factor probability is 2937, and the first 100 words of factor 1 of this document include a list of words. Unfortunately, without more context or specific details about the factor analysis, it's challenging to precisely interpret the factor and its connection to ec.autos, rec.motorcycles, rec.sport.baseball, and rec.sport.hockey.

However, by examining the word list, it appears to contain a diverse range of terms that might be associated with different topics. Some words, such as "chess," "camping," "canadiens," and "competitor," suggest a connection to sports or recreational activities. Other words like "colonial," "concession," and "contractual" seem more general and could be related to various discussions.

To generate a text that could be drawn from this factor, we can create a hypothetical paragraph:

"In the exciting world of competitive sports, where amazing feats and challenging competitions unfold, teams and players strive for victory. The chess matches on chess.ncsu.edu showcase the strategic prowess of competitors, while camping enthusiasts gather at the colonial campground for a breakaway adventure. Amid contractual negotiations and concession agreements, the Canadiens make strategic moves on the chessboard of rec.sport.hockey. The atmosphere is electric, with each participant bringing their competitive spirit to the forefront. Whether it's the adrenaline of rec.sport.baseball or the precision of chess.ncsu.edu, the diverse community engages in thrilling competitions that captivate audiences worldwide."

This paragraph attempts to incorporate some of the words from the provided list into a coherent narrative that could be related to factors learned from ec.autos, rec.motorcycles, rec.sport.baseball, and rec.sport.hockey.


The interpretation seem meaning ful, since it is related to the topic of the directory.

Exercise 6.3: Latent Dirichlet allocation

(a) Using the same data as in Exercise 6.1 (four newsgroups), create a term frequency matrix of
raw term counts for the documents.

Done above!

(b) Apply Latent Dirichlet Allocation to the term frequency matrix to find 10 underlying topic.

In [71]:
import gensim
# Create a dictionary from the documents
# startdoc=9000     # We will use the baseball n.g.
# enddoc=10000      # again, but all features
# gensim_docs=mycrawled_prunedtexts[startdoc:enddoc]
gensim_docs=mycrawled_prunedtexts
gensim_dictionary=gensim.corpora.Dictionary(gensim_docs)
# Create the document-term vectors
gensim_docvectors=[]
for k in range(len(gensim_docs)):     
    docvector=gensim_dictionary.doc2bow(gensim_docs[k])
    gensim_docvectors.append(docvector)

In [72]:
# Run the LDA optimization
numtopics=10
randomseed=124574527
numiters=10000
ninits=10
gensim_ldamodel=gensim.models.ldamodel.LdaModel( \
    gensim_docvectors, \
    id2word=gensim_dictionary,num_topics=numtopics, \
    iterations=numiters,random_state=randomseed)

(c) Describe the resulting factors: list the 10 words with highest probability in each topic. 

In [73]:
# (c) Describe the resulting factors: list the 10 words with highest probability in each topic. 
for i in range(numtopics):
    print('10 words with highest probability in topic ' + str(i+1))
    print('-------------------------------------------------------------')
    print(gensim_ldamodel.print_topic(i,topn=10))
    print('\n')

10 words with highest probability in topic 1
-------------------------------------------------------------
0.011*"edmonton" + 0.010*"buffalo" + 0.008*"giant" + 0.006*"fame" + 0.006*"draft" + 0.006*"scoring" + 0.005*"czech" + 0.004*"milwaukee" + 0.004*"ted" + 0.004*"murphy"


10 words with highest probability in topic 2
-------------------------------------------------------------
0.009*"brave" + 0.007*"captain" + 0.006*"stadium" + 0.006*"cal" + 0.005*"runner" + 0.004*"cleveland" + 0.004*"bos" + 0.004*"finish" + 0.004*"predict" + 0.004*"suck"


10 words with highest probability in topic 3
-------------------------------------------------------------
0.007*"pen" + 0.006*"jay" + 0.005*"york" + 0.005*"draft" + 0.004*"murray" + 0.004*"williams" + 0.003*"acquire" + 0.003*"frank" + 0.003*"era" + 0.003*"morris"


10 words with highest probability in topic 4
-------------------------------------------------------------
0.011*"pit" + 0.010*"mario" + 0.008*"bat" + 0.008*"bos" + 0.007*"maynard" + 

(d) Find, for each topic, the document (message) with highest probability of that topic, and print its 100 first words.

In [74]:
# Get topic content: term-topic probabilities
gensim_termtopicprobabilities=gensim_ldamodel.get_topics()
# Get topic prevalences per document, and overall topic prevalences
# (expected amount of documents per topic)
overallstrengths=numpy.zeros((numtopics,1))
documentstrengths=numpy.zeros((len(gensim_docvectors),numtopics))
for k in range(len(gensim_docvectors)):
    topicstrengths=gensim_ldamodel.get_document_topics(\
        gensim_docvectors[k],minimum_probability=0)
    for m in range(len(topicstrengths)):
        documentstrengths[k][topicstrengths[m][0]]=topicstrengths[m][1]
        overallstrengths[topicstrengths[m][0]]=\
        overallstrengths[topicstrengths[m][0]]+topicstrengths[m][1]

In [75]:
# Show top-10 words of the top topic (number 5 here)
print(gensim_ldamodel.show_topic(5,topn=10))

[('stanley', 0.01637378), ('islander', 0.015583312), ('defensive', 0.0099151675), ('offense', 0.005795607), ('giant', 0.0056868396), ('murray', 0.0051159374), ('face', 0.0046372013), ('broadcast', 0.0045163757), ('smith', 0.004167805), ('pinch', 0.0031735469)]


In [76]:
# (d) Find, for each topic, the document (message) with highest probability of that topic, and print
# its 100 first words.
for i in range(numtopics):
    print('Document index with highest topic probability: ' + str(numpy.argmax(documentstrengths[:,i]) + 1))
    print('First 100 words of topic ' + str(i+1) + ' of the document with highest probability')
    print(gensim_docs[numpy.argmax(documentstrengths[:,i])][0:100])
    print('\n')

Document index with highest topic probability: 1144
First 100 words of topic 1 of the document with highest probability
['engr.latech.edu', 'speedy', 'engr.latech.edu', 'speedy', 'mercer', 'attack', 'rabid', 'hubcap', 'yamaha', '750', 'twin', 'a.k.a', 'tank', 'wierd', 'noise', 'catch', 'glimpse', 'silver', 'left', 'foot', 'jerk', 'foot', 'peg', 'pull', 'near', 'parking', 'discover', 'type', 'hubcap', 'large', 'cage', 'hubcap', 'weigh', '4-5', 'pound', 'impact', 'bend', 'left', 'peg', 'flat', 'frame', 'shifter', 'process', 'heard', 'approach', 'cap', 'sans', 'portion', 'left', 'foot', 'sort', 'experience', 'cap', 'yard', 'delivery', 'truck', 'life', 'interest', 'several', 'sheet', 'dozen', 'gallon', 'mph', 'interesting', 'wood', 'dry', 'helmet', 'tail', 'truck', 'path', 'burst', 'impact', 'lane', '360', 'nail', 'bridge', 'half', '360', 'messy', 'diego', 'apartment', 'beach', 'sportster', 'shake', 'burger', 'fry', 'jack', 'box', 'foul', 'weather', 'jacket', 'milk', 'shake', 'open', 'blvd

(e) Do the topics seem related to individual newsgroups? Does their content seem meaningful?

Some of the topics seem related to individual newsgroups, but some of them are not. The content seems meaningful.

(f) Carry out (b)-(d) with 15 topics instead. Are some of the topics the same as before? What are the differences?

In [77]:
# Run the LDA optimization
numtopics=15
randomseed=124574527
numiters=10000
ninits=10
gensim_ldamodel=gensim.models.ldamodel.LdaModel( \
    gensim_docvectors, \
    id2word=gensim_dictionary,num_topics=numtopics, \
    iterations=numiters,random_state=randomseed)

In [78]:
# (c) Describe the resulting factors: list the 10 words with highest probability in each topic. 
for i in range(numtopics):
    print('10 words with highest probability in topic ' + str(i+1))
    print('-------------------------------------------------------------')
    print(gensim_ldamodel.print_topic(i,topn=10))
    print('\n')

10 words with highest probability in topic 1
-------------------------------------------------------------
0.017*"buffalo" + 0.014*"giant" + 0.011*"montreal" + 0.008*"scoring" + 0.007*"lineup" + 0.007*"jersey" + 0.006*"jet" + 0.006*"stats" + 0.006*"ted" + 0.005*"rockies"


10 words with highest probability in topic 2
-------------------------------------------------------------
0.019*"brave" + 0.009*"captain" + 0.008*"cal" + 0.008*"stadium" + 0.008*"atlanta" + 0.008*"runner" + 0.007*"bos" + 0.006*"3rd" + 0.006*"eric" + 0.005*"cleveland"


10 words with highest probability in topic 3
-------------------------------------------------------------
0.008*"jay" + 0.007*"murray" + 0.006*"williams" + 0.005*"draft" + 0.005*"roy" + 0.004*"acquire" + 0.004*"individual" + 0.004*"isle" + 0.004*"steal" + 0.004*"statistic"


10 words with highest probability in topic 4
-------------------------------------------------------------
0.014*"mario" + 0.010*"maynard" + 0.008*"morris" + 0.005*"=-=" + 0.005*

In [79]:
# Get topic content: term-topic probabilities
gensim_termtopicprobabilities=gensim_ldamodel.get_topics()
# Get topic prevalences per document, and overall topic prevalences
# (expected amount of documents per topic)
overallstrengths=numpy.zeros((numtopics,1))
documentstrengths=numpy.zeros((len(gensim_docvectors),numtopics))
for k in range(len(gensim_docvectors)):
    topicstrengths=gensim_ldamodel.get_document_topics(\
        gensim_docvectors[k],minimum_probability=0)
    for m in range(len(topicstrengths)):
        documentstrengths[k][topicstrengths[m][0]]=topicstrengths[m][1]
        overallstrengths[topicstrengths[m][0]]=\
        overallstrengths[topicstrengths[m][0]]+topicstrengths[m][1]

In [80]:
# Show top-10 words of the top topic (number 5 here)
print(gensim_ldamodel.show_topic(5,topn=10))

[('stanley', 0.024050219), ('islander', 0.022864446), ('defensive', 0.009085701), ('broadcast', 0.0062354463), ('giant', 0.0057611787), ('offensive', 0.005448156), ('face', 0.004192777), ('winner', 0.0033398676), ('kevin', 0.0033038915), ('champion', 0.0032330477)]


In [81]:
# (d) Find, for each topic, the document (message) with highest probability of that topic, and print
# its 100 first words.
for i in range(numtopics):
    print('Document index with highest topic probability: ' + str(numpy.argmax(documentstrengths[:,i]) + 1))
    print('First 100 words of topic ' + str(i+1) + ' of the document with highest probability')
    print(gensim_docs[numpy.argmax(documentstrengths[:,i])][0:100])
    print('\n')

Document index with highest topic probability: 607
First 100 words of topic 1 of the document with highest probability
['drutx.att.com', 'callison', 'james', 'callison', 'normally', 'unloaded', 'colt', 'delta', 'glove', 'box', 'loaded', 'magazine', 'handy', 'perfectly', 'legal', 'oklahoma', 'travel', 'inter-state', 'unloaded', 'magnum', 'revolver', 'glove', 'box', 'pocket', 'legal', 'everywhere', 'federal', 'law', 'illinois', 'police', 'hang', 'carry', 'pistol', 'load', 'unloaded', 'glove', 'compartment', 'carry', 'concealed', 'weapon', 'colorado', 'illegal', 'concealed', 'weapon', 'permit', 'unless', 'law', 'recently', 'carry', 'weapon', 'legal', 'colorado', 'conceal', 'illegal', 'newspaper', 'account', 'police', 'traffic', 'infraction', 'observe', '.357', 'magnum', 'revolver', 'sit', 'seat', 'driver', 'cite', 'carry', 'weapon', 'conceal', 'gun', 'discover', 'glove', 'box', 'crime', 'carry', 'glove', 'box', '_is_', 'legal', 'oklahoma', 'inter-state', 'travel', 'legal', 'destination', 

Some of the topics are the same as before, but some of them are different. The differences are that the topics are more specific and the words having higher probability are more related to the topic.

(g) Optional: let's see if an AI chatbot can analyze these topics. See exercise 2.4 regarding possible chatbots; report what chatbot you used. 
Take one of the topics, e.g., the highest probability one, and provide its list of top words to an AI chatbot; you can tell it that the topic has been learned from the four newsgroups. Ask it to interpret the topic based on the list of words. Ask it also to generate a text that could be drawn from the topic. Does its interpretation of the topic seem meaningful?

Question for CHATGPT:
Document index with highest topic probability: 607
First 100 words of topic 1 of the document with highest probability
['drutx.att.com', 'callison', 'james', 'callison', 'normally', 'unloaded', 'colt', 'delta', 'glove', 'box', 'loaded', 'magazine', 'handy', 'perfectly', 'legal', 'oklahoma', 'travel', 'inter-state', 'unloaded', 'magnum', 'revolver', 'glove', 'box', 'pocket', 'legal', 'everywhere', 'federal', 'law', 'illinois', 'police', 'hang', 'carry', 'pistol', 'load', 'unloaded', 'glove', 'compartment', 'carry', 'concealed', 'weapon', 'colorado', 'illegal', 'concealed', 'weapon', 'permit', 'unless', 'law', 'recently', 'carry', 'weapon', 'legal', 'colorado', 'conceal', 'illegal', 'newspaper', 'account', 'police', 'traffic', 'infraction', 'observe', '.357', 'magnum', 'revolver', 'sit', 'seat', 'driver', 'cite', 'carry', 'weapon', 'conceal', 'gun', 'discover', 'glove', 'box', 'crime', 'carry', 'glove', 'box', '_is_', 'legal', 'oklahoma', 'inter-state', 'travel', 'legal', 'destination', 'gun', 'carry', 'locked', 'glove', 'box', 'specifically', 'exclude', 'separate', 'ammo', 'legal', 'title', 'chapter', 'section', 'forget', 'copy']

tell it that the topic has been learned from the four newsgroups

interpret the topic based on the list of words.

generate a text that could be drawn from the topic


CHAT GPT RESPONSE:
It appears that the text you provided is related to discussions about the legality and regulations surrounding the transportation and carrying of firearms in different states, particularly focusing on Oklahoma, Illinois, and Colorado. The mention of terms like "unloaded," "glove box," "concealed weapon," and references to specific laws suggests a discussion on the legal nuances of carrying firearms in various locations.

Given the context and the mention of "four newsgroups," it seems like this topic has been discussed or learned from online forums or news groups where individuals share information and experiences related to firearms laws and regulations.

Here's a generated text that could be drawn from the topic:

In the realm of firearm regulations, discussions among online communities have illuminated the legal intricacies of carrying weapons across state lines. Users share insights on the permissible practices, highlighting the legality of transporting unloaded firearms in specific compartments, such as glove boxes. The discourse touches upon variations in state laws, with particular attention to the contrasting perspectives in states like Oklahoma, where carrying an unloaded firearm in a glove box is deemed legal for interstate travel, and Illinois, where law enforcement may scrutinize concealed weapons, even if unloaded.

The narrative delves into recent legal developments, shedding light on changes in Colorado's concealed carry landscape. While carrying a concealed weapon was once illegal without a permit, recent legislative shifts have rendered it permissible under certain circumstances. Anecdotes recount incidents where individuals, initially cited for traffic infractions, found themselves entangled in legal debates due to the discovery of firearms in glove boxes. The conversation underscores the importance of staying abreast of evolving laws and adhering to specific regulations governing the transportation of firearms, emphasizing the need for responsible gun ownership.

This generated text attempts to capture the essence of the topic, combining elements of legal discussions, regional variations, and the practical implications of firearm transportation.

Yes, the interpretation of the topic seems meaningful since it can recognize the reason of the topic and the generated text is related to the topic.